In [1]:
import cPickle as pickle

import scipy
import numpy as np
import pandas as pd
import re

# Load English stopwords from NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer, WhitespaceTokenizer
import nltk.data

from gensim.models import word2vec
import logging

import os

print ("Required Libraries loaded.")

Required Libraries loaded.


In [2]:
# Import articles for string cleaning

def clean_text(text, stem_words = False, remove_stopwords = False):
    ###clean
    texts = re.sub("[^a-zA-Z]",    #pattern to match
              " ",              #replace other with this
              str(text))                 #text to apply to
         
    #print "Text recived: ", texts 
    clean_corpus = texts.lower().split()
    #print "corpus: ", clean_corpus
    if stem_words:
        # Porter stemmer
        porter = nltk.PorterStemmer()
        # Snowball stemmer
        snowball = nltk.SnowballStemmer('english')
        # Lancaster stemmer
        lancaster = nltk.LancasterStemmer()
        # General stemming Lambda function to stem tokens
        clean_corpus = lambda tokens: [porter.stem(w) for w in corpus]
    if remove_stopwords:   # Optionally remove stop words
        stops = set(stopwords.words("english"))
        clean_corpus = [w for w in clean_corpus if not w in stops]
        #print "Clean_corpus: ", clean_corpus
    
    return (clean_corpus)

In [3]:
# Use the punkt tokenizer for sentence splitting

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def article_to_sentences(article, tokenizer, stem_words=False, remove_stopwords=False):
    """
    article - article to use as input to create the wordlist
    tokenizer - the tokenizer to use to split into sentences
    stem_words - boolean, whether to use the stemmer function or not
    remove_stopwords - boolean, whether to remove the stopwords function or not
    
    article_to_sentences: Function to convert a document to a sequence of sentences,
    optionally removing stop words.  
    returns: a sequence of sentences where each sentence is itself a sequence of words
    """
    raw_sentences = tokenizer.tokenize(article.strip())   # Punkt tokenize into sentences
    sentences = []   # create list of sentences 
    for raw_sentence in raw_sentences:
        #print "raw sentence: ", raw_sentence
        #print len(raw_sentence)
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(clean_text(raw_sentence, stem_words, remove_stopwords))

    # returns a list of lists: list of sentences composed of lists of words
        #print "Sentences: ", sentences
    return sentences

In [4]:
def create_sentences(corpus, name):
    print ("Begin sentences creation %s..." % name)
    sentences = []  # Initialize an empty list of sentences

    print ("Parsing sentences from training set")
    for article in corpus:
        sentences += article_to_sentences(article, tokenizer)
    print ("Finished data loading and creating sentences")
    return sentences

In [5]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
def train_save_model(sentences, filename):

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

    # Set values for various parameters
    #num_features = 210    # Word vector dimensionality                      
    min_word_count = 10   # Minimum word count                        
    num_workers = 8       # Number of threads to run in parallel
    context = 15          # Context window size                                                                                    
    downsampling = 1e-3   # Downsample setting for frequent words

    # Initialize and train the model (this will take some time)

    print ("Training model...")
    if (filename == "aceinhibitor"):
        num_features = 210
    elif (filename == "adhd"):
        num_features = 80
    elif (filename == "antihistamines"):
        num_features = 29
    elif (filename == "atypicalAntipsychotics"):
        num_features = 381
    elif (filename == "betaBlockers"):
        num_features = 194
    elif (filename == "calciumChannelBlockers"):
        num_features = 329
    elif (filename == "estrogens"):
        num_features = 233
    elif (filename == "nsaids"):
        num_features = 242
    elif (filename == "opiods"):
        num_features = 55
    elif (filename == "oralHypoglycemics"):
        num_features = 234
    elif (filename == "protonPumpInhibitors"):
        num_features = 206
    elif (filename == "skeletalMuscleRelaxants"):
        num_features = 11
    elif (filename == "statins"):
        num_features = 467
    elif (filename == "triptans"):
        num_features = 121
    elif (filename == "urinaryIncontinence"):
        num_features = 215
    model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, 
                              window = context, sample = downsampling)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)

    # It can be helpful to create a meaningful model name and 
    # save the model for later use. You can load it later using Word2Vec.load()
    
    model_name = filename + "_chi2_features_10minwords_15context"
    folder = "C:\Users\CR107\Dropbox\PhD\Experiments\TREC_2004\data\word2vec"
    print ("Saving the model: %s...." % model_name)
    model_name = os.path.join(folder, model_name)
    model.save(model_name)

    print ("Finished Model training and saving for %s...") % filename

In [6]:
#iterate through a directory, load a file, sentence = filename.TIABSMh
path = "C:\EPC_Data\TREC_BROKEN\No_Mh_Tag"
for file in os.listdir(path):
    current_file = os.path.join(path, file)
    filename, _, _ = file.split("_")
        
    file_obj = pd.read_csv(current_file, sep=",", index_col='PMID')
    corpus = file_obj.TIABSMh
   
    train_save_model(create_sentences(corpus, filename), filename)

Begin sentences creation aceinhibitor...
Parsing sentences from training set


2017-01-23 18:00:50,151 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-01-23 18:00:50,151 : INFO : collecting all words and their counts
2017-01-23 18:00:50,151 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-23 18:00:50,196 : INFO : PROGRESS: at sentence #10000, processed 253082 words, keeping 11950 word types
2017-01-23 18:00:50,230 : INFO : PROGRESS: at sentence #20000, processed 504463 words, keeping 16753 word types
2017-01-23 18:00:50,252 : INFO : collected 18630 word types from a corpus of 633837 raw words and 24993 sentences
2017-01-23 18:00:50,276 : INFO : min_count=10 retains 3900 unique words (drops 14730)
2017-01-23 18:00:50,276 : INFO : min_count leaves 599341 word corpus (94% of original 633837)
2017-01-23 18:00:50,288 : INFO : deleting the raw counts dictionary of 18630 items
2017-01-23 18:00:50,292 : INFO : sample=0.001 downsamples 56 most-common words
2017-01-23 18:00:50,292 : INFO : downsampli

Finished data loading and creating sentences
Training model...


2017-01-23 18:00:50,362 : INFO : training model with 8 workers on 3900 vocabulary and 210 features, using sg=0 hs=0 sample=0.001 negative=5
2017-01-23 18:00:50,364 : INFO : expecting 24993 sentences, matching count from corpus used for vocabulary survey
2017-01-23 18:00:51,469 : INFO : PROGRESS: at 65.50% examples, 1480399 words/s, in_qsize 0, out_qsize 0
2017-01-23 18:00:52,032 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-01-23 18:00:52,042 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-01-23 18:00:52,043 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-01-23 18:00:52,049 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-01-23 18:00:52,052 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-01-23 18:00:52,053 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-01-23 18:00:52,058 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-

Saving the model: aceinhibitor_chi2_features_10minwords_15context....
Finished Model training and saving for aceinhibitor...
Begin sentences creation adhd...
Parsing sentences from training set


...

Saving the model: triptans_chi2_features_10minwords_15context....
Finished Model training and saving for triptans...
Begin sentences creation urinaryIncontinence...
Parsing sentences from training set


2017-01-23 18:01:17,357 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-01-23 18:01:17,357 : INFO : collecting all words and their counts
2017-01-23 18:01:17,357 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-23 18:01:17,368 : INFO : collected 6502 word types from a corpus of 72639 raw words and 3126 sentences
2017-01-23 18:01:17,381 : INFO : min_count=10 retains 1004 unique words (drops 5498)
2017-01-23 18:01:17,384 : INFO : min_count leaves 59811 word corpus (82% of original 72639)
2017-01-23 18:01:17,388 : INFO : deleting the raw counts dictionary of 6502 items
2017-01-23 18:01:17,388 : INFO : sample=0.001 downsamples 56 most-common words
2017-01-23 18:01:17,391 : INFO : downsampling leaves estimated 42611 word corpus (71.2% of prior 59811)
2017-01-23 18:01:17,391 : INFO : estimated required memory for 1004 words and 215 dimensions: 2228880 bytes
2017-01-23 18:01:17,395 : INFO : resetting layer weights
2017-

Finished data loading and creating sentences
Training model...


2017-01-23 18:01:17,680 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-01-23 18:01:17,688 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-01-23 18:01:17,690 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-01-23 18:01:17,691 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-01-23 18:01:17,694 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-01-23 18:01:17,696 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-01-23 18:01:17,697 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-01-23 18:01:17,697 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-01-23 18:01:17,698 : INFO : training on 363195 raw words (213090 effective words) took 0.2s, 1081025 effective words/s
2017-01-23 18:01:17,700 : WARNING : under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-01-23 18:01:17,

Saving the model: urinaryIncontinence_chi2_features_10minwords_15context....
Finished Model training and saving for urinaryIncontinence...
